<a href="https://colab.research.google.com/github/ykumargupta/AbstractiveTextSummarizer/blob/master/TextSummarization9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

# Cleaning Dataset using Pandas and Numpy

In [3]:
import numpy as np 
import pandas as pd 

import os

In [4]:
summary = pd.read_csv('news_summary.csv', encoding='iso-8859-1')
raw = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [5]:
pre1 =  raw.iloc[:,0:2].copy()
# pre1['head + text'] = pre1['headlines'].str.cat(pre1['text'], sep =" ") 

pre2 = summary.iloc[:,0:6].copy()
pre2['text'] = pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'], sep = " "), sep =" "),sep= " "), sep = " ")

In [6]:
pre = pd.DataFrame()
pre['text'] = pd.concat([pre1['text'], pre2['text']], ignore_index=True)
pre['summary'] = pd.concat([pre1['headlines'],pre2['headlines']],ignore_index = True)
pre.head(5)

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim..."


In [7]:
pre.shape

(102915, 2)

Shape of the Dataset with NULL values *(102915, 2)*

In [8]:
pre.text.describe()

count                                                102797
unique                                               102756
top       England was beaten at home for the first time ...
freq                                                      2
Name: text, dtype: object

In [9]:
pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102915 entries, 0 to 102914
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     102797 non-null  object
 1   summary  102915 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [10]:
len(str(pre['text'].values).split()) #65 sum # 1504 text # max_text_len=100 max_summary_len=15

1504

In [11]:
pre['text'][0]

"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers."

In [12]:
pre['summary'][0]

'upGrad learner switches to career in ML & Al with 90% salary hike'

In [13]:
pre['text'][1]

"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more."

In [14]:
pre['summary'][1]

'Delhi techie wins free food from Swiggy for one year on CRED'

In [15]:
print(pre.shape)
print(pre.isnull().sum())

(102915, 2)
text       118
summary      0
dtype: int64


In [16]:
pre_clean = pre.dropna().drop_duplicates()
#pre_clean['summary'] = pre['summary'].dropna().drop_duplicates()

In [17]:
print(pre_clean.shape)
print(pre_clean.isnull().sum())

(102775, 2)
text       0
summary    0
dtype: int64


Shape of the Dataset without NULL values *(102775, 2)*

In [18]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    #text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contraction_mapping:
                new_text.append(contraction_mapping[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    return text

In [19]:
 contraction_mapping ={"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [20]:
# Clean the summaries and texts
clean_summaries = []
for summary in pre_clean.summary:
    clean_summaries.append(clean_text(summary))
print("Summaries are complete.")

clean_texts = []
for text in pre_clean.text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [21]:
clean_summaries[0]

'upGrad learner switches to career in ML & Al with 90% salary hike'

In [22]:
clean_texts[0]

"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers."

# Cleaned Dataset Samples

In [23]:
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()


Clean Review # 1
upGrad learner switches to career in ML & Al with 90% salary hike
Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.

Clean Review # 2
Delhi techie wins free food from Swiggy for one year on CRED
Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.

Clean Review # 3
New Zealand end Rohit Sharma-led India's 12-match winning streak
New Zealand defeated India by 8

# Dividing into train and test samples

In [24]:
idx = int(0.95*len(clean_texts))
print(len(clean_texts))
print(idx)
print(len(clean_texts)-idx)
train_texts = clean_texts[:idx]
test_texts = clean_texts[idx:]
train_sum = clean_summaries[:idx]
test_sum = clean_summaries[idx:]


102775
97636
5139


In [25]:
!mkdir data_test
!mkdir data_train

A subdirectory or file data_test already exists.
A subdirectory or file data_train already exists.


# Formatting of Dataset into files 

In [26]:
for i, text in enumerate(train_texts):
    file_name = './data_train/data_train_'+ str(i)+'_review.story'
    with open(file_name,'w',encoding="utf-8") as f:
        f.write(text+'\n\n@highlight\n\n'+train_sum[i])
    f.close()
    

In [27]:
for i, text in enumerate(test_texts):
    file_name = './data_test/data_test_'+ str(i)+'_review.story'
    with open(file_name,'w',encoding="utf-8") as f:
        f.write(text+'\n\n@highlight\n\n'+test_sum[i])
    f.close()

In [33]:
git clone https://github.com/ykumargupta/AbstractiveTextSummarizer.git
mv AbstractiveTextSummarizer/* .
rm -rf AbstractiveTextSummarizer
wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-02-27.zip
unzip stanford-corenlp-full-2018-02-2

SyntaxError: invalid syntax (3533529490.py, line 1)

In [2]:
!git clone https://github.com/ykumargupta/AbstractiveTextSummarizer.git
!mv AbstractiveTextSummarizer/* .
!rm -rf AbstractiveTextSummarizer
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-02-27.zip
!unzip stanford-corenlp-full-2018-02-27

'git' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [29]:
!export CLASSPATH=/content/stanford-corenlp-full-2018-02-27/stanford-corenlp-3.9.1.jar
!echo "Please tokenize this text." | java -cp stanford-corenlp-full-2018-02-27/stanford-corenlp-3.9.1.jar edu.stanford.nlp.process.PTBTokenizer

'export' is not recognized as an internal or external command,
operable program or batch file.
'java' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
!mkdir output_data
!mkdir output_data/output_train
!mkdir output_data/output_test

A subdirectory or file output_data already exists.
The syntax of the command is incorrect.
The syntax of the command is incorrect.


# This step does these :-
1. Tokenizing the dataset files 
2. Converting these tokenized files into encoded binary files
3. Chunking these files for better performance of pipeline

In [28]:
!python make_datafiles_test.py  ./data_test  ./output_data/output_test

Preparing to tokenize ./data_test to ./output_data/output_test\tokenized_stories_dir...
Making list of files to tokenize...
Tokenizing 5139 files in ./data_test and saving in ./output_data/output_test\tokenized_stories_dir...


Traceback (most recent call last):
  File "C:\Users\syeda\Final Year Project\make_datafiles_test.py", line 184, in <module>
    tokenize_stories(stories_dir, tokenized_stories_dir)
  File "C:\Users\syeda\Final Year Project\make_datafiles_test.py", line 65, in tokenize_stories
    subprocess.call(command)
  File "C:\Users\syeda\.conda\envs\tf-gpu\lib\subprocess.py", line 349, in call
    with Popen(*popenargs, **kwargs) as p:
  File "C:\Users\syeda\.conda\envs\tf-gpu\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\syeda\.conda\envs\tf-gpu\lib\subprocess.py", line 1420, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
FileNotFoundError: [WinError 2] The system cannot find the file specified


In [36]:
!python make_datafiles_train.py  ./data_train  ./output_data/output_train/

Streaming output truncated to the last 5000 lines.
./output_data/output_train/tokenized_stories_dir/data_train_89691_review.story
./output_data/output_train/tokenized_stories_dir/data_train_27352_review.story
./output_data/output_train/tokenized_stories_dir/data_train_93143_review.story
./output_data/output_train/tokenized_stories_dir/data_train_81225_review.story
./output_data/output_train/tokenized_stories_dir/data_train_32619_review.story
./output_data/output_train/tokenized_stories_dir/data_train_4120_review.story
./output_data/output_train/tokenized_stories_dir/data_train_61066_review.story
./output_data/output_train/tokenized_stories_dir/data_train_36526_review.story
./output_data/output_train/tokenized_stories_dir/data_train_55958_review.story
./output_data/output_train/tokenized_stories_dir/data_train_58126_review.story
./output_data/output_train/tokenized_stories_dir/data_train_97553_review.story
./output_data/output_train/tokenized_stories_dir/data_train_68378_review.story
./